In [1]:
import pandas as pd
import cdsapi
import netCDF4
from netCDF4 import num2date
import numpy as np
import os
import xarray as xr


In [ ]:
def download_climate_data_xarray(year, month, day, latitude, longitude, area, day_night_ind, result_df):
    area_str = '_'.join(str(num) for num in area)
    c = cdsapi.Client()
    file_name = './historical_climate_data/weather-'+area_str+'-'+str(year)+'-'+str(month)+'-'+str(day)+'.nc'
    time = '12:00'
    if day_night_ind == "N":
        time = '22:00'
    c.retrieve('reanalysis-era5-single-levels',
    {
        'product_type': 'reanalysis',
        'format': 'netcdf',
        'variable': [
            '10m_u_component_of_wind', '10m_v_component_of_wind', '2m_temperature',
            'soil_temperature_level_1', 'soil_temperature_level_2', 'soil_temperature_level_3',
            'soil_temperature_level_4', 'soil_type', 'total_precipitation', 'volumetric_soil_water_layer_1'
        ],
        'year': year,
        'month': month,
        'day': [str(day)],
        'time': [time],
        'area': area,
    },
    file_name)

    ds = xr.open_dataset(file_name)

    longitude = np.mean(ds['longitude'].values)
    latitude = np.mean(ds['latitude'].values)
    u10 = np.mean(ds['u10'].values)
    v10 = np.mean(ds['v10'].values)
    t2m = np.mean(ds['t2m'].values)
    stl1 = np.mean(ds['stl1'].values)
    stl2 = np.mean(ds['stl2'].values)
    stl3 = np.mean(ds['stl3'].values)
    stl4 = np.mean(ds['stl4'].values)
    slt = np.mean(ds['slt'].values)
    tp = np.mean(ds['tp'].values)
    vswl1 = 0

    new_row = {'longitude':longitude, 
           'latitude':latitude, 
           '10m_u_component_of_wind': u10,
                   '10m_v_component_of_wind': v10,
                   '2m_temperature': t2m,
                   'soil_temperature_level_1': stl1,
                   'soil_temperature_level_2': stl2,
                   'soil_temperature_level_3': stl3,
                   'soil_temperature_level_4': stl4,
                   'soil_type': slt,
                   'total_precipitation': tp,
                   'volumetric_soil_water_layer_1': vswl1}
    result_df.loc[len(result_df)] = new_row
    

In [ ]:
def group_fire_data_xarray(file_name):
    file_path = "./historical_fire_data/" + file_name + ".csv"
    fire_data = pd.read_csv(file_path)
    
    filtered_fire_data = fire_data.query("`confidence` >= 75 and `type` == 0")
    grouped_fire_data = filtered_fire_data.groupby(['acq_date', 'daynight']).agg({'latitude':'mean','longitude':'mean','brightness':'mean','confidence':'mean', 'frp':'mean', 'bright_t31':'mean'}).reset_index()

    #grouped_fire_data.apply(apply_on_each_group)

    result_df = pd.DataFrame({'longitude': pd.Series(dtype='float'),
                   'latitude': pd.Series(dtype='float'),
                   '10m_u_component_of_wind': pd.Series(dtype='float'),
                   '10m_v_component_of_wind': pd.Series(dtype='float'),
                   '2m_temperature': pd.Series(dtype='float'),
                   'soil_temperature_level_1': pd.Series(dtype='float'),
                   'soil_temperature_level_2': pd.Series(dtype='float'),
                   'soil_temperature_level_3': pd.Series(dtype='float'),
                   'soil_temperature_level_4': pd.Series(dtype='float'),
                   'soil_type': pd.Series(dtype='float'),
                   'total_precipitation': pd.Series(dtype='float'),
                   'volumetric_soil_water_layer_1': pd.Series(dtype='float')})
    
    #print(grouped_fire_data)
    for modis_data_index, modis_row in grouped_fire_data.iterrows():
        #print(modis_row['acq_date'])
        date_value = modis_row['acq_date']
        date_split = date_value.split("-")
        #print(type(weather_row['DATE']))
        year = date_split[0]
        month = date_split[1]
        day = date_split[2]
        #print(year + month + day)
        confidence = modis_row['confidence']
        latitude = modis_row['latitude']
        longitude = modis_row['longitude']
        day_night_ind = modis_row['daynight']
        #print(type(confidence))
        if confidence > 75:
            north_coord = latitude+0.25
            south_coord = latitude-0.25
            west_coord = longitude-0.25
            east_coord = longitude+0.25
            area = [north_coord, west_coord, south_coord, east_coord]
            #print(area)
            download_climate_data(year, month, day, latitude, longitude, area, day_night_ind, result_df)
    
    result_filename = "./historical_climate_data/xarray/"+file_name+".csv"
    result_df.to_csv(result_filename)
    

In [2]:
def download_climate_data(year, month, day, area, day_night_ind):
    area_str = '_'.join(str(num) for num in area)
    c = cdsapi.Client()
    file_name = './historical_climate_data/weather-'+area_str+'-'+str(year)+'-'+str(month)+'-'+str(day)+'.nc'
    time = '12:00'
    if day_night_ind == "N":
        time = '22:00'
    c.retrieve('reanalysis-era5-single-levels',
    {
        'product_type': 'reanalysis',
        'format': 'netcdf',
        'variable': [
            '10m_u_component_of_wind', '10m_v_component_of_wind', '2m_temperature',
            'soil_temperature_level_1', 'soil_temperature_level_2', 'soil_temperature_level_3',
            'soil_temperature_level_4', 'soil_type', 'total_precipitation',
        ],
        'year': year,
        'month': month,
        'day': [str(day)],
        'time': [time],
        'area': area,
    },
    file_name)

    nc = netCDF4.Dataset(file_name, mode='r')
    print(nc.variables['t2m'][:])
    cols = list(nc.variables.keys())
    list_nc = []
    for c in cols:
        list_nc.append(list(nc.variables[c][:]))
    df_nc = pd.DataFrame(list_nc)
    df_nc = df_nc.T
    df_nc.columns = cols
    csv_file_name = './historical_climate_data/csv/weather-'+area_str+'-'+str(year)+str(month)+'-'+str(day)+'.csv'
    df_nc.to_csv(csv_file_name, index = False)

In [3]:
#download_climate_data(2022, 1, 11, [59.7686, -125.092, 49.7686, -115.092], 'D')

In [4]:
def download_satellite_image(year, month, day, area, day_night_ind):
    print("image")

In [5]:
def get_climate_data_for_fire_data(file_name):
    file_path = "./historical_fire_data/" + file_name
    fire_data = pd.read_csv(file_path)
    for modis_data_index, modis_row in fire_data.iterrows():
        date_value = modis_row['acq_date']
        date_split = date_value.split("-")
        #print(type(weather_row['DATE']))
        year = date_split[0]
        month = date_split[1]
        day = date_split[2]
        #print(year + month + day)
        confidence = modis_row['confidence']
        latitude = modis_row['latitude']
        longitude = modis_row['longitude']
        day_night_ind = modis_row['daynight']
        #print(type(confidence))
        if confidence > 70:
            north_coord = latitude+0.25
            south_coord = latitude-0.25
            west_coord = longitude-0.25
            east_coord = longitude+0.25
            area = [north_coord, west_coord, south_coord, east_coord]
            #print(area)
            download_climate_data(year, month, day, area, day_night_ind)

In [6]:
#get_climate_data_for_fire_data('canada_2023.csv')

In [7]:
def group_fire_data(file_name):
    file_path = "./historical_fire_data/" + file_name
    fire_data = pd.read_csv(file_path)
    
    filtered_fire_data = fire_data.query("`confidence` >= 75 and `type` == 0")
    grouped_fire_data = filtered_fire_data.groupby(['acq_date', 'daynight']).agg({'latitude':'mean','longitude':'mean','brightness':'mean','confidence':'mean', 'frp':'mean', 'bright_t31':'mean'}).reset_index()

    #grouped_fire_data.apply(apply_on_each_group)
    
    #print(grouped_fire_data)
    for modis_data_index, modis_row in grouped_fire_data.iterrows():
        #print(modis_row['acq_date'])
        date_value = modis_row['acq_date']
        date_split = date_value.split("-")
        #print(type(weather_row['DATE']))
        year = date_split[0]
        month = date_split[1]
        day = date_split[2]
        #print(year + month + day)
        confidence = modis_row['confidence']
        latitude = modis_row['latitude']
        longitude = modis_row['longitude']
        day_night_ind = modis_row['daynight']
        #print(type(confidence))
        if confidence > 75:
            north_coord = latitude+0.25
            south_coord = latitude-0.25
            west_coord = longitude-0.25
            east_coord = longitude+0.25
            area = [north_coord, west_coord, south_coord, east_coord]
            #print(area)
            download_climate_data(year, month, day, area, day_night_ind)
                

#print(daynight)
    #return grouped_fire_data



In [ ]:
group_fire_data("canada_2023.csv")

2024-01-15 17:47:42,400 INFO Welcome to the CDS
2024-01-15 17:47:42,402 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2024-01-15 17:47:42,514 INFO Request is queued
2024-01-15 17:54:01,581 INFO Request is completed
2024-01-15 17:54:01,584 INFO Downloading https://download-0015-clone.copernicus-climate.eu/cache-compute-0015/cache/data8/adaptor.mars.internal-1705341122.9465227-9646-1-2e7ab8e9-b966-4f23-810d-6c1ed22e69f0.nc to ./historical_climate_data/weather-53.336800000000004_-117.56796666666666_52.836800000000004_-117.06796666666666-2023-01-01.nc (3.1K)
2024-01-15 17:54:01,942 INFO Download rate 8.8K/s                                                                                                                                                                                                 
2024-01-15 17:54:02,088 INFO Welcome to the CDS
2024-01-15 17:54:02,089 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/

[[[262.85845947 262.81658988]
  [261.89551937 262.57500655]
  [258.88763428 259.57887646]]]


2024-01-15 17:58:20,792 INFO Request is completed
2024-01-15 17:58:20,798 INFO Downloading https://download-0005-clone.copernicus-climate.eu/cache-compute-0005/cache/data6/adaptor.mars.internal-1705341488.8430274-29887-7-3ec4131d-b9a1-466e-b293-b9ea2ac83133.nc to ./historical_climate_data/weather-54.66256666666666_-117.794_54.16256666666666_-117.294-2023-01-01.nc (3.1K)
2024-01-15 17:58:21,075 INFO Download rate 11.3K/s                                                                                                                                                                                                
2024-01-15 17:58:21,257 INFO Welcome to the CDS
2024-01-15 17:58:21,258 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels


[[[271.9979248  271.35511484 270.24795532]
  [271.88641041 271.74998153 271.1808202 ]]]


2024-01-15 17:58:21,459 INFO Request is queued
2024-01-15 18:02:40,067 INFO Request is completed
2024-01-15 18:02:40,076 INFO Downloading https://download-0005-clone.copernicus-climate.eu/cache-compute-0005/cache/data5/adaptor.mars.internal-1705341692.4000833-31654-6-8f3dfbc4-d117-4689-9211-f3b7b8fa3490.nc to ./historical_climate_data/weather-53.7462_-118.216975_53.2462_-117.716975-2023-01-02.nc (3.2K)
2024-01-15 18:02:40,361 INFO Download rate 11.3K/s                                                                                                                                                                                                
2024-01-15 18:02:40,528 INFO Welcome to the CDS
2024-01-15 18:02:40,529 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels


...

[[[266.15417608 265.83959961]
  [267.58128549 267.29064133]
  [268.30169678 268.14356321]]]


2024-01-15 19:14:14,795 INFO Request is queued
2024-01-15 19:16:08,781 INFO Request is completed
2024-01-15 19:16:08,782 INFO Downloading https://download-0013-clone.copernicus-climate.eu/cache-compute-0013/cache/data4/adaptor.mars.internal-1705346139.5946498-30286-1-c9e4bf43-fed6-4696-acf8-1cc2ee28b303.nc to ./historical_climate_data/weather-52.0199_-121.5835_51.5199_-121.0835-2023-01-19.nc (3.2K)
2024-01-15 19:16:09,115 INFO Download rate 9.6K/s                                                                                                                                                                                                 
2024-01-15 19:16:09,249 INFO Welcome to the CDS
2024-01-15 19:16:09,251 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2024-01-15 19:16:09,376 INFO Request is queued


[[[271.25558556 270.75556219 270.16342163]
  [271.55735335 271.37189491 271.29818336]
  [271.1594178  271.49312281 271.62191772]]]


2024-01-15 19:22:28,424 INFO Request is completed
2024-01-15 19:22:28,429 INFO Downloading https://download-0009-clone.copernicus-climate.eu/cache-compute-0009/cache/data0/adaptor.mars.internal-1705346539.6966646-24078-2-87fb9685-42b6-4b28-b6e1-0e7263b1828b.nc to ./historical_climate_data/weather-56.27814285714286_-111.29634285714285_55.77814285714286_-110.79634285714285-2023-01-20.nc (3.2K)
2024-01-15 19:22:28,762 INFO Download rate 9.6K/s                                                                                                                                                                                                 
2024-01-15 19:22:28,905 INFO Welcome to the CDS
2024-01-15 19:22:28,906 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2024-01-15 19:22:29,023 INFO Request is queued


[[[266.83263328 267.13306427 266.96424049]
  [266.74183655 267.13067033 267.06905463]
  [266.9566527  266.8676768  266.76953704]]]


In [ ]:
file_path = "./historical_fire_data/canada_2023.csv" 
fire_data = pd.read_csv(file_path)
fire_data.dtypes